# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [3]:
# TODOS: 
# 1) import any other libraries you might need
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum
from pyspark.sql import Window
import datetime
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

# 2) instantiate a Spark session 
spark = SparkSession \
    .builder \
    .appName('Wrangling quiz') \
    .getOrCreate()

# 3) read in the data set located at the path "data/sparkify_log_small.json"
path = 'data/sparkify_log_small.json'
user_log = spark.read.json(path)
# 4) write code to answer the quiz questions 

# Question 1

Which page did user id "" (empty string) NOT visit?

In [27]:
user_log.select('page').dropDuplicates().show()
user_log.printSchema()
user_log.take(1)

+----------------+
|            page|
+----------------+
|Submit Downgrade|
|            Home|
|       Downgrade|
|          Logout|
|   Save Settings|
|           About|
|        Settings|
|           Login|
|        NextSong|
|            Help|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string 

[Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046')]

In [6]:
# TODO: write your code to answer question 1
empty_user = user_log.filter(user_log.userId == '') \
    .select('page') \
    .alias('blank_pages')\
    .dropDuplicates()
all_pages = user_log.select('page').dropDuplicates()

print('Pages not visited:')
for row in set(all_pages.collect()) - set(empty_user.collect()):
    print(row.page)
   
print('\nPages visited:')
for row in set(empty_user.collect()):
    print(row.page)

SyntaxError: invalid syntax (<ipython-input-6-141c7d2883d6>, line 7)

# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [2]:
# TODO: use this space to explore the behavior of the user with an empty string
'''Someone without an account, they dont have access to anything such as settings or anything a memeber
free or paid would.'''

# Question 3

How many female users do we have in the data set?

In [7]:
# TODO: write your code to answer question 3
females = user_log.filter(user_log.gender == 'F') \
    .select('userId') \
    .dropDuplicates()
females.count()

462

# Question 4

How many songs were played from the most played artist?

In [26]:
# TODO: write your code to answer question 4
user_log.filter(user_log.page == 'NextSong') \
    .select('Artist') \
    .groupBy('Artist') \
    .agg({'Artist' : 'count'}) \
    .sort(desc('count(Artist)')) \
    .withColumnRenamed('count(Artist)', 'Play_count') \
    .show(1)


+--------+----------+
|  Artist|Play_count|
+--------+----------+
|Coldplay|        83|
+--------+----------+
only showing top 1 row



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [33]:
# TODO: write your code to answer question 5
home_page = user_log.filter(user_log.page == 'Home')
home_page.filter(home_page.song)

AnalysisException: "filter expression '`song`' of type string is not a boolean.;;\nFilter song#19: string\n+- Filter (page#16 = Home)\n   +- Relation[artist#6,auth#7,firstName#8,gender#9,itemInSession#10L,lastName#11,length#12,level#13,location#14,method#15,page#16,registration#17L,sessionId#18L,song#19,status#20L,ts#21L,userAgent#22,userId#23] json\n"

In [37]:
function = udf(lambda ishome : int(ishome == 'Home'), IntegerType())

user_window = Window \
    .partitionBy('userID') \
    .orderBy(desc('ts')) \
    .rangeBetween(Window.unboundedPreceding, 0)

cusum = user_log.filter((user_log.page == 'NextSong') | (user_log.page == 'Home')) \
    .select('userID', 'page', 'ts') \
    .withColumn('homevisit', function('page')) \
    .withColumn('period', Fsum('homevisit').over(user_window))

cusum.filter((cusum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .agg({'count(period)':'avg'}).show()

+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+

